In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import pickle
import sklearn
from datetime import datetime
import torch

package_path = Path.cwd().parent
if package_path not in sys.path:
    sys.path.append(str(package_path))

from utils.dataset_class import Phase3DataSet, Setting
from phase3.scripts.custom_pipeline import get_model_pkl, dump_model_pkl
from models.meta.platipus_class import Platipus
from hpc_scripts.param_generator import get_all_params
from models.meta.init_params import init_params

In [2]:
x_stateset = {}
rvol_raw = {}
ss_reagent_vols = {}
all_params = get_all_params(cross_validate=False)
params = all_params[172]
amine = ['JMXLWMIFDJCGBV-UHFFFAOYSA-N']
train_params = {}
platipus = {}
device = {}

for i in range(2):
    x_stateset[i] = np.genfromtxt(f'./statesets/scaled_stateset_{datetime.today().date()}.csv', delimiter=',')
    rvol_raw[i] = pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')
    ss_reagent_vols[i] = rvol_raw[i][[col for col in rvol_raw[i].columns if 'Reagent' in col]] 
    
    train_params[i] = init_params(params)
    train_params[i]['gpu_id'] = i
    platipus[i] = Platipus(train_params[i], amine=amine,
                            model_name=train_params[i]['model_name'],
                            epoch_al=True)
    platipus[i].load_model('./plat_trained_model/testing/drp_chem_10shot_4000.pkl')
    platipus[i].current_epoch = 4000
    device[i] = torch.device(f"cuda:{i}")


Meta learning rate = 0.001
Getting FULL dataset
Holding out ['CALQKRVFTWDYDG-UHFFFAOYSA-N', 'KOAGKPNEVYEZDU-UHFFFAOYSA-N', 'FCTHQYIDLRRROX-UHFFFAOYSA-N', 'JMXLWMIFDJCGBV-UHFFFAOYSA-N']
Number of reactions in training set 1477
Number of successful reactions in the training set 215
Generating training batches
Generating testing batches for testing! DO NOT RUN IF YOU SEE THIS LINE!
Meta learning rate = 0.001
Getting FULL dataset
Holding out ['CALQKRVFTWDYDG-UHFFFAOYSA-N', 'KOAGKPNEVYEZDU-UHFFFAOYSA-N', 'FCTHQYIDLRRROX-UHFFFAOYSA-N', 'JMXLWMIFDJCGBV-UHFFFAOYSA-N']
Number of reactions in training set 1477
Number of successful reactions in the training set 215
Generating training batches
Generating testing batches for testing! DO NOT RUN IF YOU SEE THIS LINE!


In [6]:
"""
query_indices_set0 = [10022, 1441]
query_indices_set1 = [17833, 9275]
query_instances_set0 = []
query_instances_set1 = []
for idx in query_indices_set0:
    query_instances_set0.append(x_stateset_set0[idx])
for idx in query_indices_set1:
    query_instances_set1.append(x_stateset_set0[idx])
"""

In [3]:
results = {}
results[0] = {10022:1, 1441:0, 6127:0, 9716:0, 8205:0, 7177:0, 17269:1, 15434:0}
results[1] = {17833:0, 9275:0, 6470:0, 2378:1, 6890:0, 17171:0, 16771:0, 14533:0}

xss_torch0 = {}
xss_torch1 = {}
x_t = {}
y_t = {}
for i in range(2):
    iters, all_data, all_labels, x_train, y_train, x_v, y_v = platipus[i].setup_active_learning(i, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')
    x_t[i] = x_train
    y_t[i] = y_train
    xss_torch0[i] = torch.from_numpy(x_stateset[i][:10000]).float().to(device[i])
    xss_torch1[i] = torch.from_numpy(x_stateset[i][10000:]).float().to(device[i])
    for exp_num in results[i]:
        
        if exp_num >= 10000:
            index = exp_num-10000
            x_t[i] = torch.cat((x_t[i], xss_torch1[i][index].view(1, 50)))
            y_t[i] = torch.cat((y_t[i], torch.from_numpy(np.array([results[i][exp_num]])).to(device[i])))
            xss_torch1[i] = torch.cat([xss_torch1[i][0:index], xss_torch1[i][index + 1:]])
            ss_reagent_vols[i].drop(ss_reagent_vols[i].index[exp_num], inplace=True)
        else:
            index = exp_num
            x_t[i] = torch.cat((x_t[i], xss_torch0[i][exp_num].view(1, 50)))
            y_t[i] = torch.cat((y_t[i], torch.from_numpy(np.array([results[i][exp_num]])).to(device[i])))
            xss_torch0[i] = torch.cat([xss_torch0[i][0:index], xss_torch0[i][index + 1:]])
            ss_reagent_vols[i].drop(ss_reagent_vols[i].index[exp_num], inplace=True)

/home1/06453/vshekar/.local/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
value0, index0 = platipus[0].phase3_active_learning(x_t[0], y_t[0], xss_torch0[0], debug=True)
print(index0.item(), value0.item())
del value0
torch.cuda.empty_cache()

tensor([0.5842, 0.5747, 0.5755,  ..., 0.5419, 0.5446, 0.7224], device='cuda:0',
       grad_fn=<MaxBackward0>)
4486 0.5386906862258911


In [5]:
value0, index0 = platipus[0].phase3_active_learning(x_t[0], y_t[0], xss_torch1[0], debug=True)
print(index0.item(), value0.item())
del value0
torch.cuda.empty_cache()

tensor([0.6885, 0.6861, 0.6835,  ..., 0.5955, 0.5983, 0.6026], device='cuda:0',
       grad_fn=<MaxBackward0>)
34 0.5341784954071045


In [6]:
value0, index0 = platipus[1].phase3_active_learning(x_t[1], y_t[1], xss_torch0[1], debug=True)
print(index0.item(), value0.item())
del value0
torch.cuda.empty_cache()

tensor([0.6863, 0.7189, 0.7133,  ..., 0.5801, 0.5749, 0.6548], device='cuda:1',
       grad_fn=<MaxBackward0>)
5179 0.5000090003013611


In [7]:
value0, index0 = platipus[1].phase3_active_learning(x_t[1], y_t[1], xss_torch1[1], debug=True)
print(index0.item(), value0.item())
del value0
torch.cuda.empty_cache()

tensor([0.6557, 0.6498, 0.6421,  ..., 0.6461, 0.6424, 0.6400], device='cuda:1',
       grad_fn=<MaxBackward0>)
3920 0.5000097751617432


In [11]:
import random
random.randint(0, 20000)

17168

In [7]:
value0, index0 = platipus_set0.phase3_active_learning(x_t, y_t, xss_torch0)

In [8]:
print(value0, index0)

tensor(0.5261, device='cuda:0', grad_fn=<MinBackward0>) tensor(9716, device='cuda:0')


In [9]:
del value0
del index0
torch.cuda.empty_cache()

In [6]:
del xss_torch0
torch.cuda.empty_cache()

In [10]:
#xss_torch1 = torch.from_numpy(x_stateset_set0[10000:]).float().to(device_set0)
value1, index1 = platipus_set0.phase3_active_learning(x_t, y_t, xss_torch1)

In [11]:
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(nan, device='cuda:0', grad_fn=<MinBackward0>) tensor(0, device='cuda:0')


In [8]:
ss_reagent_vols[0].iloc[10034]

Reagent1 (ul)     59
Reagent2 (ul)    320
Reagent3 (ul)     31
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)     90
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 10040, dtype: int64

In [9]:
ss_reagent_vols[1].iloc[5179]

Reagent1 (ul)    156
Reagent2 (ul)     20
Reagent3 (ul)    164
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    160
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 5180, dtype: int64

In [9]:
#value, index = platipus.phase3_active_learning(x_t, y_t, xss_torch)
value, index

(tensor(0.5000, device='cuda:1', grad_fn=<MinBackward0>),
 tensor(7833, device='cuda:1'))

In [7]:
x_t = torch.cat((x_t, xss_torch[7833].view(1, 50)))

In [8]:
y_t = torch.cat((y_t, torch.from_numpy(np.array([0])).to(device)))

In [9]:
y_t.shape

torch.Size([11])

In [10]:
value, index = platipus.phase3_active_learning(x_t, y_t, xss_torch)

In [11]:
print(value, index)

tensor(0.5556, device='cuda:1', grad_fn=<MinBackward0>) tensor(9275, device='cuda:1')


In [16]:
set_id = 1 
iters, all_data, all_labels, x_t, y_t, x_v, y_v = platipus[0].setup_active_learning(set_id, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')

In [18]:
xss_torch0 = torch.from_numpy(x_stateset[0][:10000]).float().to(device[0])
value0, index0 = platipus[0].phase3_active_learning(x_t, y_t, xss_torch0)

In [19]:
print(value0, index0)
del value0
torch.cuda.empty_cache()

tensor(0.5000, device='cuda:0', grad_fn=<MinBackward0>) tensor(8205, device='cuda:0')


In [19]:
xss_torch1 = torch.from_numpy(x_stateset_set1[10000:]).float().to(device_set1)
value1, index1 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch1)

In [20]:
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(0.5000, device='cuda:1', grad_fn=<MinBackward0>) tensor(6125, device='cuda:1')


In [20]:
rvol_raw =  pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')
ss_reagent_vols = rvol_raw[[col for col in rvol_raw.columns if 'Reagent' in col]]
ss_reagent_vols.iloc[8205]

Reagent1 (ul)     46
Reagent2 (ul)     20
Reagent3 (ul)    324
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    110
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 8205, dtype: int64

## PLATIPUS SET 1

In [13]:
set_id = 1
iters, all_data, all_labels, x_t, y_t, x_v, y_v = platipus_set1.setup_active_learning(set_id, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')

In [14]:
xss_torch0 = torch.from_numpy(x_stateset_set0[:10000]).float().to(device_set1)
xss_torch1 = torch.from_numpy(x_stateset_set0[10000:]).float().to(device_set1)

In [15]:
results = {
    17833:0,
    9275:0,
    6470:0,
}
for exp_num in results:
    if exp_num >= 10000:
        x_t = torch.cat((x_t, xss_torch1[exp_num-10000].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set1)))
    else:
        x_t = torch.cat((x_t, xss_torch1[exp_num].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set1)))

In [16]:
value0, index0 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch0)
print(value0, index0)
del value0
del index0
torch.cuda.empty_cache()

tensor(0.5018, device='cuda:1', grad_fn=<MinBackward0>) tensor(2378, device='cuda:1')


In [17]:
value1, index1 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch1)
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(0.6820, device='cuda:1', grad_fn=<MinBackward0>) tensor(589, device='cuda:1')


In [18]:
ss_reagent_vols_set0.iloc[2378]

Reagent1 (ul)      6
Reagent2 (ul)    280
Reagent3 (ul)      4
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    210
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 2378, dtype: int64